## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server
MLflow Tracking e MLflow Registry

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='novo_experimento', tags={}>]

In [11]:
client.create_experiment(name="new-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [3]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [4]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: afdeb09bca434c00849f37b33ab4fa39, rmse: 6.3052
run id: 8f6d67be52df4da987f3cd4f6938ec95, rmse: 6.3534
run id: d14f447fdc8142a3857904d130a0f238, rmse: 6.4592
run id: 84566f946c0a4776a52e9310a2eb3799, rmse: 6.4592
run id: a16d8e5fdb1840dd9254dfa4e03dc7e0, rmse: 6.4592


### Interacting with the Model Registry

In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [6]:
run_id = "84566f946c0a4776a52e9310a2eb3799"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/07/25 10:40:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 6
Created version '6' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1658756414825, current_stage='None', description=None, last_updated_timestamp=1658756414825, name='nyc-taxi-regressor', run_id='84566f946c0a4776a52e9310a2eb3799', run_link=None, source='./mlruns/1/84566f946c0a4776a52e9310a2eb3799/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [8]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Production
version: 3, stage: Staging
version: 6, stage: None


In [9]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1658676408829, current_stage='Staging', description=('The model version 2 was transitioned to Staging on 2022-07-24\n'
 '\n'
 'Foi pra Produção no dia 25/07/2022'), last_updated_timestamp=1658756489635, name='nyc-taxi-regressor', run_id='8a06ee5de98f4789baa8d3d21823dd2e', run_link='', source='./mlruns/1/8a06ee5de98f4789baa8d3d21823dd2e/artifacts/model', status='READY', status_message=None, tags={'model': 'LinearSVR'}, user_id=None, version=2>

In [10]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1658676408829, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2022-07-25', last_updated_timestamp=1658756517276, name='nyc-taxi-regressor', run_id='8a06ee5de98f4789baa8d3d21823dd2e', run_link='', source='./mlruns/1/8a06ee5de98f4789baa8d3d21823dd2e/artifacts/model', status='READY', status_message=None, tags={'model': 'LinearSVR'}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [12]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [13]:
df = read_dataframe("../data/green_tripdata_2021-03.parquet")

In [ ]:
client.download_artifacts(run_id=run_id, path='../preprocessor', dst_path='.')

In [14]:
import pickle

with open("../preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [15]:
X_test = preprocess(df, dv)

In [16]:
target = "duration"
y_test = df[target].values

In [17]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

Wall time: 1.09 s


{'rmse': 973.4296949331057}

In [18]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

Wall time: 1.24 s


{'rmse': 6.659623830022514}

In [19]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1658676412872, current_stage='Production', description='', last_updated_timestamp=1658756684407, name='nyc-taxi-regressor', run_id='b01fa6f234974d0882d6bcab539af6b0', run_link='', source='./mlruns/1/b01fa6f234974d0882d6bcab539af6b0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>